In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 80

In [4]:
train_dir = '/content/drive/MyDrive/VIPERC/dataset/train'
test_dir = '/content/drive/MyDrive/VIPERC/dataset/test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 5025 images belonging to 13 classes.
Found 1256 images belonging to 13 classes.


In [5]:
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

for layer in base_model.layers:
    layer.trainable = False

74836368/74836368 [==============================] - 0s 0us/step


In [6]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

In [9]:
model.fit(train_generator, epochs=EPOCHS, validation_data=test_generator, callbacks=[checkpoint, early_stopping])

Epoch 1/80
158/158 [==============================] - 1718s 11s/step - loss: 0.2360 - accuracy: 0.9325 - val_loss: 0.0491 - val_accuracy: 0.9889
Epoch 2/80
158/158 [==============================] - 35s 220ms/step - loss: 0.0323 - accuracy: 0.9916 - val_loss: 0.1705 - val_accuracy: 0.9475
Epoch 3/80
158/158 [==============================] - 30s 191ms/step - loss: 0.0214 - accuracy: 0.9946 - val_loss: 0.0984 - val_accuracy: 0.9682
Epoch 4/80
158/158 [==============================] - 30s 191ms/step - loss: 0.0240 - accuracy: 0.9926 - val_loss: 0.1166 - val_accuracy: 0.9642
Epoch 5/80
158/158 [==============================] - 37s 233ms/step - loss: 0.0238 - accuracy: 0.9916 - val_loss: 0.0474 - val_accuracy: 0.9896
Epoch 6/80
158/158 [==============================] - 34s 212ms/step - loss: 0.0146 - accuracy: 0.9966 - val_loss: 0.0432 - val_accuracy: 0.9889
Epoch 7/80
158/158 [==============================] - 33s 211ms/step - loss: 0.0068 - accuracy: 0.9980 - val_loss: 0.0351 - val_ac

In [11]:
!mv /content/best_model.keras /content/drive/MyDrive/VIPERC/

In [12]:
from tensorflow.keras.models import load_model

best_model = load_model('/content/drive/MyDrive/VIPERC/best_model.keras')

In [13]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

class_labels = list(test_generator.class_indices.keys())
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_labels, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

40/40 [==============================] - 12s 148ms/step - loss: 0.0321 - accuracy: 0.9936

Test Loss: 0.0321
Test Accuracy: 99.36%
40/40 [==============================] - 10s 164ms/step
Accuracy: 99.36305732484077
Precision: 99.36597104980432
Recall: 99.36305732484077
F1 Score: 99.36302088082623

Classification Report:
              precision    recall  f1-score   support

 cong_chieng     1.0000    1.0000    1.0000        60
     dan_bau     0.9875    1.0000    0.9937        79
      dan_co     0.9787    0.9787    0.9787        47
      dan_da     1.0000    0.9837    0.9918       123
     dan_day     1.0000    1.0000    1.0000        92
  dan_nguyet     0.9840    0.9840    0.9840       125
     dan_sen     0.9847    0.9847    0.9847       131
  dan_t_rung     1.0000    0.9942    0.9971       171
    dan_tinh     1.0000    1.0000    1.0000       124
   dan_tranh     0.9880    1.0000    0.9939       164
   dan_ty_ba     1.0000    1.0000    1.0000        48
        khen     1.0000    1.